In [3]:
%load_ext autoreload
%autoreload 2
import numpy as np
from src import GetUserClimbData as gud
from src import AssesItemBased as assess
from src.UserClimbCollaborativeFiltering import pairwise_pearson_correlation
from sklearn.metrics.pairwise import pairwise_distances, cosine_similarity
import pandas as pd

In [7]:
number_cols = ['id','latitude','longitude','clean_pitches','clean_rating','mean_type_ord']
route_feats[number_cols].shape

(3590, 6)

In [2]:
uid_path = './user_ids/uids.csv'
user_data = gud.get_users_climbs(uid_path)
user_data_idx = user_data.set_index('user_id')

In [3]:
route_data = user_data.T  
route_data = route_data.drop(labels='user_id', axis=0)


In [78]:
test_split = user_data_idx[:6]
train_split = user_data_idx[6:]
        
held_out_routes, test_trim = assess.trim_routes(test_split,0.2)
train_held_out = train_split.append(test_trim)

In [5]:
cos_sim = cosine_similarity(route_data)
corr_sim = pairwise_distances(X=route_data.values,metric='correlation',n_jobs=6)
jacc_sim = pairwise_distances(X=route_data.values,metric='jaccard',n_jobs=6)

In [47]:
assess.test_folds(user_data_idx,cos_sim,folds=5,name='cos_20_',cutoff=20)

In [48]:
metrics = assess.run_eval('cos_20_',5)
full_name = 'item_based_cos_sim_cutoff_20'
mf = pd.DataFrame([metrics[1]+[full_name]],columns=metrics[0]+['name'])
mf.to_csv('./analyze/{}.csv'.format(full_name),index=False)

In [49]:
mf

,Mean num_ret,Std num_ret,Mean num_rel,Std num_rel,Mean num_rel_ret,Std num_rel_ret,Mean num_unjug_ret@20,Std num_unjug_ret@20,Mean map,Std map,...,Std P30,Mean P100,Std P100,Mean P200,Std P200,Mean P500,Std P500,Mean P1000,Std P1000,name
0,120.0,0.0,8.8,6.04649,0.0,0.0,120.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,item_based_cos_sim_cutoff_20


In [27]:
assess.test_folds(user_data_idx,cos_sim,5,'cos_20_',10)
metrics = assess.run_eval('cos_10_',5)
mf = pd.DataFrame([metrics[1]],columns=metrics[0])
mf.to_csv('./analyze/item_based_cos_sim_cutoff_10.csv',index=False)

In [25]:
assess.test_folds(user_data_idx,corr_sim,5,'corr_',20)
metrics = assess.run_eval('corr_',5)
mf = pd.DataFrame([metrics[1]],columns=metrics[0])
mf.to_csv('./analyze/item_based_corr_sim_cutoff_20.csv',index=False)

In [26]:
assess.test_folds(user_data_idx,jacc_sim,5,'jacc_',20)
metrics = assess.run_eval('jacc_',5)
mf = pd.DataFrame([metrics[1]],columns=metrics[0])
mf.to_csv('./analyze/item_based_jacc_sim_cutoff_20.csv',index=False)

In [28]:
user_data_avg = gud.get_users_climbs(uid_path)
user_data_avg_idx = user_data_avg.set_index('user_id')

In [31]:
route_data_avg = user_data_avg.T  
route_data_avg = route_data_avg.drop(labels='user_id', axis=0)

In [32]:
cos_sim_avg = cosine_similarity(route_data_avg)
corr_sim_avg = pairwise_distances(X=route_data_avg.values,metric='correlation',n_jobs=6)
jacc_sim_avg = pairwise_distances(X=route_data_avg.values,metric='jaccard',n_jobs=6)

In [34]:
assess.test_folds(user_data_avg_idx,cos_sim_avg,5,'avg_cos_10_',10)
metrics = assess.run_eval('avg_cos_10_',5)
mf = pd.DataFrame([metrics[1]],columns=metrics[0])
mf.to_csv('./analyze/item_based_avg_rating_cos_sim_cutoff_10.csv',index=False)

In [35]:
assess.test_folds(user_data_avg_idx,cos_sim_avg,5,'avg_cos_20_',20)
metrics = assess.run_eval('avg_cos_20_',5)
mf = pd.DataFrame([metrics[1]],columns=metrics[0])
mf.to_csv('./analyze/item_based_avg_rating_cos_sim_cutoff_20.csv',index=False)

In [26]:
from src import RequestMPData as rmp
from src import RecPopular as rp


In [45]:
pd.DataFrame([metrics[1]],columns=metrics[0])

,Mean MAP,Std MAP,Mean NDCG10,Std NDCG10
0,0.006,0.0053,0.1408,0.0486


In [48]:
mf = rp.paralell_pop(user_data_idx,[10,50,100],[0.5,1,2])
mf

,Mean MAP,Std MAP,Mean NDCG10,Std NDCG10,starWeight,voteWeight
0,0.00888,0.00881,0.13238,0.07430,10,0.5
0,0.00700,0.00500,0.13309,0.10394,10,1.0
0,0.00650,0.00473,0.17601,0.08987,10,2.0
0,0.00467,0.00239,0.13963,0.05867,50,0.5
0,0.00399,0.00171,0.15351,0.08641,50,1.0
0,0.00513,0.00659,0.12775,0.04932,50,2.0
0,0.00588,0.00474,0.13152,0.08002,100,0.5
0,0.00435,0.00289,0.16357,0.07748,100,1.0
0,0.00346,0.00147,0.14389,0.11204,100,2.0
